In [1]:
import pandas as pd 
import numpy as np
from os import listdir
from os.path import isfile, join
import plotly.express as px
import pytz
eastern = pytz.timezone('US/Eastern')
import yfinance 

from dotmap import DotMap
import yaml
config = DotMap(yaml.safe_load(open("src/config.yaml")), _dynamic=False)

In [2]:
def filter_trading_hours(df, time_column):
    T = df[time_column].dt
    min_mask = (T.hour >= 10) | ((T.hour == 9) & (T.minute >= 31))
    max_mask = (T.hour < 16) | ((T.hour == 16) & (T.minute <= 1))
    return df.loc[min_mask & max_mask, :]

In [3]:
def preprocess_iq_feed_prices(prices: pd.DataFrame) -> pd.DataFrame: 
    if "time" in prices.columns:
        # Intra-day data
        prices.loc[:, "time"] = prices.loc[:, "time"].dt.tz_localize(None)
        prices.loc[:, "time"] = prices.loc[:, "time"].dt.tz_localize(eastern)
        prices.drop_duplicates(keep="first", inplace=True)
        prices.dropna(inplace=True)
        
        prices = filter_trading_hours(df=prices, time_column="time")

        # Deals with duplicate rows which occurr when not all the digits for volume are 
        # correctly entered, but only the first 1-3. So keep the largest.
        prices = prices.sort_values(["time", "volume"], ascending=[True, False])
        prices = prices.drop_duplicates(subset=["time"], keep="first")

        prices.set_index("time", inplace=True)
        prices.sort_index(ascending=True, inplace=True)
        assert prices.index.is_unique
    else:
        # Daily data
        prices.dropna(inplace=True)
        prices["date"] = pd.to_datetime(prices.date)
    return prices

In [269]:
onlyfiles = [f for f in listdir(config.data.iqfeed.minute.raw) if isfile(join(config.data.iqfeed.minute.raw, f))]
tickers = [x.split("_")[0] for x in onlyfiles]
ticker = "MBGH"

In [270]:
prices: pd.DataFrame = preprocess_iq_feed_prices(pd.read_parquet(path=f"{config.data.iqfeed.minute.raw}/{ticker}_1min.parquet", 
                                                                 columns=["time", "close", "open", "volume"]))
prices_daily: pd.DataFrame = preprocess_iq_feed_prices(pd.read_parquet(path=f"{config.data.iqfeed.minute.raw}/daily/{ticker}_daily.parquet", 
                                                                       columns=["date", "close", "open", "volume"]))

In [271]:
df = yfinance.download(ticker, period="15y",actions=True)

[*********************100%***********************]  1 of 1 completed


# Adjust for Splits and Dividends

In [221]:
intra_closes = prices.loc[(prices.index.hour == 16) & (prices.index.minute == 1), "open"]
# Converts datetime[us] to datetime[ns] and floors to daily frequency.
intra_closes.index = pd.to_datetime(intra_closes.index.date)
intra_closes.index.name = "date"
intra_closes.name = "close_intra"

In [222]:
merged = pd.merge(intra_closes, prices_daily[["date", "close"]], left_on="date", right_on="date", suffixes=("_intra", "_eod"))

In [223]:
merged = merged.sort_index(ascending=False)
merged["split_ratio"] = 1

In [167]:
merged.rename(columns=dict(close="close_eod"), inplace=True)

In [195]:
# Calculate splits (IQFeed EOD isplit adjusted, but not dividend adjusted)
# To adjust, we go backwrds in time. 
# When a split has occurred the close will be the first price that is influenced in the historical adjusted time series.
# Assuming the split/dividend has occurred overnight.
for i in range(len(merged.index)):
    row = merged.index[i]
    
    split_ratios = merged.iloc[i:i+5]["close_eod"] / merged.iloc[i:i+5]["close_intra"]
    
    # All split ratios are larger than 1% => persistent discrepancy
    if min(np.abs(split_ratios - 1)) >= 0.01:
        split_ratio = merged.loc[row, "close_eod"] / merged.loc[row, "close_intra"]
        merged.loc[row:, "close_intra"]  = (merged.loc[row:, "close_intra"] * split_ratio)
        merged.loc[row, "split_ratio"]  = split_ratio

In [197]:
X = merged.date.dt.tz_localize("US/Eastern")
_merged_intra_idx = X  + pd.DateOffset(hours=16, minutes=1)

In [198]:
prices["split_ratio"] = 1
prices.loc[_merged_intra_idx, "split_ratio"] = merged["split_ratio"]
prices["cum_split_ratio"] = np.cumprod(prices["split_ratio"][::-1])[::-1] 

In [ ]:
# split_ratio < 1 => price goes down
# stocks dont trade in fractions of a penny, hence we need to round to 2 decimals
prices[["open", "close"]] = (prices[["open", "close"]] * prices["cum_split_ratio"])

# split_ratio < 1 => volume goes up, since price goes down and pricevolume has to stay the same 
# (volume denotes number of stocks traded)
prices["volume"] = prices["volume"] / prices["cum_split_ratio"]

In [ ]:
#px.line(merged[["close_intra", "close_eod"]])

# Check quality looking at the amount of splits/dividends

In [231]:
# Download yfinance split data and check if there is a difference between the calculated splits/dividends
# IMPORTANT !!!!!!!!!!
# In yahoo finance the `Close` is adj. for splits and the `Adj. Close` for splits and dividends
# Dividends on Yahoo Finance are adjusted for splits!
import yfinance

df=yfinance.download(ticker, period="10y",actions=True)

[*********************100%***********************]  1 of 1 completed


In [211]:
df["split_ratio"] = 1
split_mask = df["Stock Splits"] > 0
# TODO Do we have to apply the split the next day or the current day?
df.loc[split_mask, "split_ratio"] = 1 / df.loc[split_mask, "Stock Splits"] 

#df["Dividends"] = df["Dividends"].shift(-1, fill_value=0)
dividend_mask = (df["Dividends"] > 0)
df.loc[dividend_mask, "split_ratio"] = 1 + df.loc[dividend_mask, "Dividends"] / df.loc[dividend_mask, "Close"]

In [212]:
df.loc[:, "split_ratio"] = df.loc[:, "split_ratio"].shift(-1)

In [ ]:
# Check quality looking at the difference between intra_day adj open and eod adj open.
# This difference should be smaller than the split_ratio for all days where the split_ratio is not 1
# This difference should also be smaller in general than our threshold for splits (1%?)

In [ ]:
# Once this has been checked and intraday data consistency/continuity is assured we could
# Replace all the closes and opens from the intra day data set with those of the eod data set.
# However this leads to problems, if we calculate wrong a single split event.
# If we don't do this replacement only one bday is affected (split day). 
# If we do the replacement multiple days will be affected by the error.

# Save adjusted files to disk